# 数字识别-图片识别

## 1.导入所需模块

In [ ]:
from nxbot import Robot,event,bgr8_to_jpeg
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
import cv2
import numpy as np
from PIL import Image
from IPython.display import display
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
import net
import matplotlib.pyplot as plt
import threading
import queue

## 2.实例化模型

In [ ]:
net = net.Net()

## 3.加载模型

In [ ]:
# 选择默认的模型或者自己训练的模型
model_path = r'../../../models/local/personal_net/digit_classification.pth'
# model_path = 'studens_models/MNIST_student.pth'
model = torch.load(model_path)
if torch.cuda.is_available():
    model = model.cuda()

## 4.对图片进行滤波处理

1. 将图片处理成灰度图
2. 通过滤波处理找出图片中物体的轮廓

In [ ]:
# 二值化滤波，返回滤波后的图片和轮廓数量
def get_img_contour_thresh(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 高斯阈值
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    ret, thresh_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # 返回轮廓列表contours（ndarray）；  
    '''
    第二个参数表示轮廓的检索模式：
        cv2.RETR_EXTERNAL表示只检测外轮廓
        cv2.RETR_LIST检测的轮廓不建立等级关系
        cv2.RETR_CCOMP建立两个等级的轮廓，上面的一层为外边界，里面的一层为内孔的边界信息。如果内孔内还有一个连通物体，这个物体的边界也在顶层。
        cv2.RETR_TREE建立一个等级树结构的轮廓。

    第三个参数method为轮廓的近似办法
        cv2.CHAIN_APPROX_NONE存储所有的轮廓点，相邻的两个点的像素位置差不超过1，即max（abs（x1-x2），abs（y2-y1））==1
        cv2.CHAIN_APPROX_SIMPLE压缩水平方向，垂直方向，对角线方向的元素，只保留该方向的终点坐标，例如一个矩形轮廓只需4个点来保存轮廓信息
        cv2.CHAIN_APPROX_TC89_L1，CV_CHAIN_APPROX_TC89_KCOS使用teh-Chinl chain 近似算法
    '''   
    contours, _ = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return contours, thresh_img

## 5.图片预处理

In [ ]:
# 图片预处理
def preprogress(thresh_img):
    
    newImage = np.array(thresh_img)
    newImage = newImage.reshape(28, 28, 1)
    transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))])
    newImage = transform(newImage)
    newImage = torch.unsqueeze(newImage,0).cuda().float()
    return newImage

## 6.检测模型是否能正常使用

我们通过numpy创建与我们将要预测的图片格式一致的形状为（28，28，1）的数组，这里我们创建的全为1的数组将这个数组经过预处理再将数据放入模型中，如果能运行通过说明模型可以正常使用了。

In [ ]:
try:
    img_data = np.ones([28,28,1],np.float32)
    model(preprogress(img_data))
except:
    print('请检查模型是否正确')

## 7.创建显示窗口界面

In [ ]:
image_widget = widgets.Image(format='jpeg')
thresh_widget = widgets.Image(format='jpeg')
result_info = widgets.Textarea(
    placeholder='NXROBO',
    description='预测结果',
    disabled=False
)

## 8.开始预测

In [ ]:
global detect_flag
detect_flag = True
global number
number=None
def detection():
    while detect_flag:
        
        time.sleep(0.03)
        img = rbt.camera.read()
        if img is not None:
            # 找出图像中的轮廓，contours表示轮廓列表，list中每个元素都是图像中的一个轮廓，用numpy中的ndarray表示，thresh_img轮廓图像。
            contours, thresh_img = get_img_contour_thresh(img)
            thresh_img = cv2.resize(thresh_img, (28, 28))
            # 如果有轮廓    
            if len(contours) > 0:
                # 找出最大轮廓
                contour = max(contours, key=cv2.contourArea)
                # 轮廓大小
                contour_area = cv2.contourArea(contour)

                # 设定轮廓阈值
                if (1000 < contour_area < 30000):
                    # 数据预处理
                    newImage = preprogress(thresh_img)
                    # 开始预测
                    with torch.no_grad():
                        out = model(newImage)
                        prob, index = torch.max(out, 1)
                        if prob > 0.8:
                            global number
                            number = index.item()
                            cv2.putText(img, "number is: " + str(number), (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)


            img = cv2.resize(img, (320, 240))
            image_widget.value = bgr8_to_jpeg(img)

            thresh_img = cv2.resize(thresh_img, (320, 240))
            thresh_widget.value = bgr8_to_jpeg(thresh_img)
            
# 创建线程
process1 = threading.Thread(target=detection,)

## 9.使用线程说出预测结果

In [ ]:
def interaction():
    global detect_flag
    while detect_flag:
        time.sleep(0.05)
        global number
        if number is not None:
            result_info.value = '预测结果是数字：{}'.format(number)
            rbt.speech.play_text('预测结果是数字{}'.format(number), True)
            number = None
# 创建线程
process2 = threading.Thread(target=interaction,)

## 10.开始预测

In [ ]:
rbt = Robot()
rbt.connect()
rbt.camera.start()
rbt.base.set_ptz(20)

process1.start()
process2.start()

display(ipywidgets.HBox([image_widget, thresh_widget]))
display(result_info)

## 11.断开连接

In [ ]:
# detect_flag=False
# rbt.disconnect()